In [ ]:
import wandb
wandb.login()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import copy

In [ ]:
# MODEL-M1
class M1(nn.Module):
    def __init__(self):
        super(M1, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=(5,5))
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5,5))
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)            

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# CIFAR10 dataset

# Transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform = transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform = transform)

# Train-val split
torch.manual_seed(39)
val_size = int(0.1*len(train_dataset))
train_size = len(train_dataset) - val_size
train, val = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [ ]:
# Dataloader & Hyperparameters
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
batch_size = 8
learning_rate = 1e-3
epochs = 50

M2 = M1().to(device)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=4)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def train(train_loader, val_loader, model, amount, device=device):  
  
  wandb.init(
    project = 'VCL_Tasks',
    group = 'Task_5',
    job_type = f'Sparsity:{amount}'
    )
  
  # Load pruning masks from Task 3
  checkpoint = torch.load(f'save_model/Task_3/M2_sparsity_{amount*100}.tar', map_location=device)
  weight_mask = {n.split('_')[0]: torch.abs(w) < 1e-6 for n, w in checkpoint['weight_mask']}
  
  # We load M1 model weights here 
  model.load_state_dict(torch.load('save_model/Task_1/M1_weights.pth', map_location=device))

  criterion = nn.CrossEntropyLoss().to(device=device)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
  for epoch in range(epochs):
    # Training
    avg_train_loss = 0
    train_acc = []
    model.train() 
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(device=device)
        label = label.to(device=device)
        
        out = model(image)
        loss = criterion(out, label)
        acc = accuracy(out, label)
        train_acc.append(acc)
        avg_train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        
        # zero the gradients of interest
        for name, module in model.named_parameters():  
            if module.grad is not None and name in weight_mask:
                module.grad[weight_mask[name]] = 0 
        
        
        optimizer.step()
        
    avg_train_loss = avg_train_loss/len(train_loader)
    avg_train_acc = torch.stack(train_acc).mean()
    wandb.log({'Task_5/Train Accuracy': avg_train_acc, 'Task_5/epoch':epoch+1})
    wandb.log({'Task_5/Train Loss':avg_train_loss, 'Task_5/epoch':epoch+1})


    # Validation 
    avg_val_loss = 0
    val_acc = []
    model.eval()
    for batch_idx, (image, label) in enumerate(val_loader):
        with torch.no_grad():
            image = image.to(device=device)
            label = label.to(device=device)

            out = model(image)
            loss = criterion(out, label)
            acc = accuracy(out, label)
            val_acc.append(acc)

            avg_val_loss += loss.item()
        
    avg_val_loss = avg_val_loss/len(val_loader)
    avg_val_acc = torch.stack(val_acc).mean()
    wandb.log({'Task_5/Validation Accuracy' : avg_val_acc, 'Task_5/epoch':epoch+1})
    wandb.log({'Task_5/Validation Loss' : avg_val_loss, 'Task_5/epoch':epoch+1})

    print(f'Epochs:{epoch+1}, Average Train Loss:{avg_train_loss}, Average Validation Loss:{avg_val_loss}, Validation Accuracy:{avg_val_acc}')

    torch.save(model.state_dict(), f'save_model/Task_5/model_retrained_weights_sparsity_{amount*100}.pth')
    wandb.save(f'save_model/Task_5/model_retrained_weights_sparsity_{amount*100}.pth')
  wandb.finish()  

In [ ]:
for amount in np.arange(4, 10, 1)/10:
    train(train_loader, val_loader, M2, amount)